In [162]:
import numpy as np
import matplotlib.pyplot as plt

In [163]:
from loaders import clean_text_for_pattern_letter
from temp_text_puh import template_str
# input_str = clean_text_for_pattern_letter(template_str, out_len=len(template_str))
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
input_str = '<' + input_str + '>'
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
           'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', '\'', '>']
print(f'{letters=}')
# print(f'{input_str=}')

len(letters)=32
502
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '<', "'", '>']


In [164]:
# generate SDR

out_len = 128 # 128
sparsity = 4 # 4
max_overlap = 1 # 1

sparse_prob = sparsity / out_len

letter_SDRs = {}

for letter in letters:
    while True:
        rnd_SDR = np.random.choice([0, 1], out_len, True, [1 - sparse_prob, sparse_prob])

        if (rnd_SDR > 0).sum() != sparsity:
            continue
        
        is_over_max = False
        for inp_SDR in letter_SDRs.values():
            if (inp_SDR & rnd_SDR).sum() > max_overlap:
                is_over_max = True
                break
        
        if is_over_max:
            continue
            
        break
    
    # print(f'{letter}')
    letter_SDRs.setdefault(letter, rnd_SDR)

for letter, inp_SDR in letter_SDRs.items():
    print(f'{letter} : {(inp_SDR > 0).sum()} {inp_SDR.tolist()}')

a : 4 [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
b : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
c : 4 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0

In [165]:
# text to SDRs
input_SDRs = []

for inp in input_str:
    input_SDRs.append(letter_SDRs[inp])

print(f'{len(input_SDRs)=}')

# for SDR in input_SDRs:
#     print(f'{SDR.tolist()}')

len(input_SDRs)=502


In [166]:
# init start column
column_size = 16

start_SDR = letter_SDRs['<']

start_I = np.zeros((start_SDR.size, column_size))

active_start_SDR_idx = np.where(start_SDR == 1)

for as_idx in active_start_SDR_idx[0]:
    rnd_active = np.random.randint(column_size)

    start_I[as_idx][rnd_active] = 1

# for col in np.rot90(start_I):
#     print(f'{col.astype(int).tolist()}')

In [167]:
from numpy import ndarray


def train(input_SDRs: list[ndarray],
          push_delta=0.04,
          epoch_cnt=1,
          inp_len=128,
          out_len=128,
          sparsity=4,
          eps0=0.01,
          eps0_V=0.09,
          ):
    I = np.zeros(inp_len)
    K = np.zeros(out_len)

    S = np.random.rand(out_len, out_len) * 1e-2

    V = np.zeros(out_len)

    prec = 1e-30

    inputs_idx = np.arange(len(input_SDRs))

    for epoch_idx in range(epoch_cnt):
        np.random.shuffle(inputs_idx)

        # eps = (epoch_cnt - epoch_idx) * eps0

        for SDR_idx in inputs_idx:
            I = np.copy(input_SDRs[SDR_idx])

            K = np.dot(S, I)

            K /= np.max(np.abs(K)) + prec

            nV = np.abs(V - np.max(V))
            nV /= np.max(nV) + prec

            OV = K + nV * eps0_V

            max_O_idx = np.argsort(OV)[::-1][:sparsity]
            pull_idx = max_O_idx[0]
            push_idx = max_O_idx[1:sparsity]

            # eps
            eps = np.zeros(K.shape)
            eps[max_O_idx] = eps0 * ((np.linalg.norm(S[max_O_idx], axis=1) - 1)**2)

            S[pull_idx] += (I - S[pull_idx] * K[pull_idx][np.newaxis].T) * eps[pull_idx][np.newaxis].T
            S[push_idx] += (I - S[push_idx] * K[push_idx][np.newaxis].T) * (-push_delta) * eps[push_idx][np.newaxis].T

            # winners
            V[pull_idx] += 1

    print(np.sum(V == 0))

    return S, V


S, UsV = train(input_SDRs=input_SDRs,
          epoch_cnt=100,
          push_delta=0.04,
          out_len=128,
          sparsity=4,
          eps0=0.0001,
          eps0_V=0.09,)
print(f'{S.max()=} {S.min()=} {np.mean(S)=}')


94
S.max()=0.3044299378699078 S.min()=-0.010152739721564821 np.mean(S)=0.005456536421120189


In [168]:
def infer_one(S: ndarray, inp_SDR: ndarray, sparsity=4) -> tuple[ndarray, ndarray]:
    K = np.dot(S, inp_SDR)

    max_O_idx = np.argsort(K)[::-1][:sparsity]

    BK = np.zeros(K.shape)
    BK[max_O_idx] = 1

    return K, BK

for inp_SDR in input_SDRs:
    O, BK = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

    print(BK.astype(int).tolist())

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0

In [169]:
prec = 1e-30

V = np.zeros((out_len, column_size))

def train_predict(input_SDRs: list[ndarray],
                  S: ndarray,
                  V: ndarray,
                  start_I: ndarray,
                  out_len,
                  column_size,
                  prec=1e-30,
                  epoch_cnt=1,
                  sparsity=4,):
    I = np.zeros((out_len, column_size))
    O = np.zeros((out_len, column_size))

    W = np.random.rand(out_len, column_size, out_len * column_size) * 1e-2

    eps0 = 0.003

    eps0_V = 0.09

    for epoch_idx in range(epoch_cnt):
        I = np.copy(start_I)

        for inp_SDR in input_SDRs[1:]:
            K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

            I = I.reshape(out_len * column_size)

            O = np.dot(W, I)

            O /= np.max(np.abs(O)) + prec

            nV = np.abs(V - np.max(V, axis=1)[np.newaxis].T)
            nV /= np.max(nV, axis=1)[np.newaxis].T + prec

            OV = O + nV * eps0_V

            OV *= SDR[np.newaxis].T
            max_O = np.max(OV, axis=1)

            O *= SDR[np.newaxis].T
            O[OV < max_O[np.newaxis].T] = 0
            max_O_idx = np.where(O != 0)

            W[max_O_idx] += (I - W[max_O_idx] * O[max_O_idx][..., np.newaxis]) * eps0

            # winners
            V[max_O_idx] += 1

            # recur
            O[O != 0] = 1

            I = np.copy(O)

    # O.shape
    print(np.mean(V, axis=1).tolist())
    print(np.sum(V == 0))
    print(np.sum((V > 0), axis=1).tolist())
    # for col in np.rot90(V):
    #     print(f'{col.astype(int).tolist()}')

    return W, V


W, V = train_predict(input_SDRs=input_SDRs,
                     S=S,
                     V=V,
                     start_I=start_I,
                     epoch_cnt=1,
                     prec=prec,
                     out_len=out_len,
                     column_size=column_size,
                     sparsity=sparsity,)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.6875, 5.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 6.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.625, 0.0, 0.0, 3.8125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.125, 0.0, 8.125, 0.0, 5.6875, 0.0, 0.0, 0.0, 0.0, 8.125, 5.1875, 0.0, 0.0, 0.0, 0.8125, 0.0, 0.0, 0.0, 0.0, 0.4375, 8.875, 0.0, 0.0, 0.0, 0.0, 3.5, 0.0, 0.0, 0.0, 8.3125, 0.0, 3.9375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.1875, 0.5625, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.9375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.625, 0.0, 6.875, 6.75, 0.0, 5.6875, 0.0, 0.0, 0.0, 0.0]
1689
[0, 0, 0, 0, 0, 0, 0, 14, 16, 4, 0, 0, 0, 0, 0, 8, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 16, 0, 16, 0, 0, 0, 0, 16, 16, 0, 0, 0, 11, 0, 0, 0, 0, 6, 16, 0, 0, 0, 0, 16, 0, 0, 0, 16, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 6, 0, 0, 0, 0, 0, 0

In [170]:
# test

def test_predict(S: ndarray, W: ndarray, input_SDRs: list[ndarray], start_I: ndarray, sparsity=4):
    error = []

    I = np.copy(start_I)

    for inp_SDR in input_SDRs[1:]:
        K, SDR = infer_one(S=S, inp_SDR=inp_SDR, sparsity=sparsity)

        I = I.reshape(out_len * column_size)

        O = np.dot(W, I)

        max_O = np.max(O, axis=1)
        argmax_O_idx = np.argsort(max_O)[::-1][:sparsity]

        SDR_O = np.zeros(O.shape[0])
        SDR_O[argmax_O_idx] = 1

        err = (SDR.astype(int) ^ SDR_O.astype(int)).sum()
        # print(f'{err=}')
        error.append(err)

        O *= SDR[np.newaxis].T
        max_O = np.max(O, axis=1)
        O[O < max_O[np.newaxis].T] = 0

        # recur
        O[O != 0] = 1

        I = np.copy(O)

    error = np.array(error)
    mean_percent = int(np.mean(error) / (2 * sparsity) * 100)
    print(f'{np.mean(error)=} {error.max()=} {error.min()=} {mean_percent=}')

    return error


test_predict(S=S,
             W=W,
             input_SDRs=input_SDRs,
             start_I=start_I,
             sparsity=sparsity,)

# O.shape
# for col in np.rot90(O):
#     print(f'{col.astype(int).tolist()}')


np.mean(error)=1.5489021956087825 error.max()=8 error.min()=0 mean_percent=19


array([2, 2, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 4, 2, 0, 4, 0,
       2, 0, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 0, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 2, 6, 0, 2, 2, 2, 0, 4, 4,
       6, 4, 0, 2, 2, 0, 2, 0, 0, 0, 6, 2, 0, 4, 2, 0, 2, 4, 4, 0, 4, 2,
       4, 0, 2, 2, 2, 4, 2, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 2,
       4, 0, 4, 0, 4, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 4, 0, 4, 0,
       2, 4, 0, 2, 0, 4, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 6, 0, 2, 0, 0, 0,
       2, 4, 0, 2, 2, 2, 2, 4, 2, 2, 0, 0, 0, 0, 6, 2, 2, 2, 0, 2, 0, 0,
       2, 0, 6, 0, 2, 2, 2, 2, 2, 4, 0, 0, 0, 0, 2, 2, 0, 0, 0, 2, 6, 2,
       0, 0, 0, 2, 4, 6, 0, 2, 0, 2, 0, 4, 2, 2, 2, 0, 0, 4, 2, 0, 0, 0,
       2, 2, 0, 0, 4, 0, 2, 2, 2, 0, 2, 4, 0, 0, 0, 4, 2, 4, 0, 2, 0, 2,
       2, 6, 0, 2, 4, 0, 2, 0, 4, 2, 0, 0, 0, 4, 0, 0, 6, 2, 2, 4, 0, 2,
       2, 2, 2, 0, 0, 6, 0, 0, 6, 0, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 6, 2,
       2, 2, 0, 2, 4, 0, 0, 0, 0, 4, 2, 2, 0, 0, 0,

In [171]:
def after_train_one(K: ndarray,
                    S: ndarray,
                    dS: ndarray,
                    V: ndarray,
                    I: ndarray,
                    push_delta=0.04,
                    eps0=0.0001,
                    sparsity=4,
                    eps0_V=0.09,):
    K /= np.max(np.abs(K)) + prec

    nV = np.abs(V - np.max(V))
    nV /= np.max(nV) + prec

    OV = K + nV * eps0_V

    max_O_idx = np.argsort(OV)[::-1][:sparsity]
    pull_idx = max_O_idx[0]
    push_idx = max_O_idx[1:sparsity]

    # eps
    eps = np.zeros(K.shape)
    eps[max_O_idx] = eps0 * ((np.linalg.norm(S[max_O_idx], axis=1) - 1)**2)

    S[pull_idx] += (I - S[pull_idx] * K[pull_idx][np.newaxis].T) * eps[pull_idx][np.newaxis].T
    S[push_idx] += (I - S[push_idx] * K[push_idx][np.newaxis].T) * (-push_delta) * eps[push_idx][np.newaxis].T

    # winners
    V[pull_idx] += 1

    return dS


In [204]:
# after train
dS = np.zeros(S.shape)
aS = np.copy(S)
aW = np.copy(W)
aV = np.copy(V)
aUsV = np.copy(UsV)

I = np.copy(start_I)
prev_SDR = np.copy(start_SDR)
prev_K, k_SDR = infer_one(S=aS, inp_SDR=prev_SDR, sparsity=sparsity)
prev_K /= np.max(np.abs(prev_K)) + prec

preference_feedback = 0.3

for inp_SDR in input_SDRs[1:]:
    K, SDR = infer_one(S=aS, inp_SDR=inp_SDR, sparsity=sparsity)

    I = I.reshape(out_len * column_size)

    O = np.dot(aW, I)

    max_O = np.max(O, axis=1)
    max_O /= np.max(np.abs(max_O)) + prec
    OK = np.copy(K)
    K /= np.max(np.abs(K)) + prec
    aK = K + max_O * preference_feedback

    # clustering train
    dS = after_train_one(K=aK, S=aS, dS=dS, V=aUsV, I=inp_SDR, eps0=0.0001, sparsity=sparsity)

    # next I
    O *= SDR[np.newaxis].T
    max_O = np.max(O, axis=1)
    O[O < max_O[np.newaxis].T] = 0

    O[O != 0] = 1
    
    OK = (OK + np.zeros(O.shape).T).T
    # print(f'{OK.shape=}')

    # reverse input
    rvs_I = np.dot(aW.T.reshape(aW.shape[0], aW.shape[1], aW.shape[2]), OK.flatten())
    rvs_I = np.max(rvs_I, axis=1)
    # print(f'{rvs_I.shape=}')
    # print(f'{rvs_I.max()=} {rvs_I.min()=} {np.mean(rvs_I)=}')

    rvs_I /= np.max(np.abs(rvs_I)) + prec
    prev_K += rvs_I * preference_feedback
    
    # prev clustering train
    dS = after_train_one(K=prev_K, S=aS, dS=dS, V=aUsV, I=prev_SDR, eps0=0.0001, sparsity=sparsity)

    # recur
    # O[O != 0] = 1

    I = np.copy(O)

    prev_SDR = np.copy(inp_SDR)
    prev_K = np.copy(K)


In [205]:
a = np.arange(8)
b = np.zeros((8, 2))
c = (a + b.T).T
c.astype(int)

array([[0, 0],
       [1, 1],
       [2, 2],
       [3, 3],
       [4, 4],
       [5, 5],
       [6, 6],
       [7, 7]])

In [206]:
w = np.arange(3*2*6).reshape(3, 2, 6)
print('w', w)
print()
print('w.T', w.reshape(3, 2, 3, 2).transpose(2, 3, 0, 1).reshape(3, 2, 6))
print('w.R')
print(w.T.reshape(3, 2, 6))


w [[[ 0  1  2  3  4  5]
  [ 6  7  8  9 10 11]]

 [[12 13 14 15 16 17]
  [18 19 20 21 22 23]]

 [[24 25 26 27 28 29]
  [30 31 32 33 34 35]]]

w.T [[[ 0  6 12 18 24 30]
  [ 1  7 13 19 25 31]]

 [[ 2  8 14 20 26 32]
  [ 3  9 15 21 27 33]]

 [[ 4 10 16 22 28 34]
  [ 5 11 17 23 29 35]]]
w.R
[[[ 0 12 24  6 18 30]
  [ 1 13 25  7 19 31]]

 [[ 2 14 26  8 20 32]
  [ 3 15 27  9 21 33]]

 [[ 4 16 28 10 22 34]
  [ 5 17 29 11 23 35]]]


In [207]:
aW, aV = train_predict(input_SDRs=input_SDRs,
                       #    S=(S + dS),
                       S=aS,
                       V=np.zeros((out_len, column_size)),
                       start_I=start_I,
                       epoch_cnt=1,
                       prec=prec,
                       out_len=out_len,
                       column_size=column_size,
                       sparsity=sparsity,)

test_predict(S=aS,
             # S=(S + dS),
             W=aW,
             input_SDRs=input_SDRs,
             start_I=start_I,
             sparsity=sparsity,)


[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.6875, 5.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 6.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.8125, 0.0, 0.0, 8.625, 0.0, 0.0, 3.8125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3.125, 0.0, 8.125, 0.0, 5.6875, 0.0, 0.0, 0.0, 0.0, 7.875, 5.1875, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.875, 0.4375, 0.0, 0.0, 0.0, 3.5, 0.0, 0.0, 0.0, 8.3125, 0.0, 3.9375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 4.1875, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 9.9375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 6.625, 0.5625, 6.875, 6.75, 0.0, 5.6875, 0.0, 0.0, 0.0, 0.0]
1686
[0, 0, 0, 0, 0, 0, 0, 16, 16, 0, 4, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 0, 11, 0, 0, 16, 0, 0, 16, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 15, 0, 16, 0, 16, 0, 0, 0, 0, 16, 16, 0, 8, 6, 0, 0, 0, 0, 0, 0, 16, 5, 0, 0, 0, 16, 0, 0, 0, 16, 0, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 16, 0, 0, 0, 0, 0, 0, 

array([0, 0, 0, 2, 2, 2, 0, 4, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 0, 4,
       0, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0,
       2, 2, 2, 2, 0, 4, 2, 0, 0, 0, 6, 2, 0, 0, 4, 0, 0, 2, 2, 2, 0, 2,
       2, 0, 2, 2, 0, 0, 2, 4, 0, 0, 4, 2, 0, 0, 4, 2, 2, 0, 2, 2, 4, 0,
       2, 2, 0, 0, 2, 2, 0, 4, 2, 0, 4, 2, 2, 0, 4, 0, 0, 4, 0, 2, 2, 0,
       0, 2, 2, 4, 0, 4, 2, 2, 4, 0, 0, 0, 2, 0, 2, 0, 0, 0, 6, 0, 0, 0,
       2, 2, 4, 2, 0, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 2, 4, 2, 0, 4, 0, 4,
       2, 2, 2, 0, 2, 2, 2, 0, 2, 0, 0, 0, 2, 0, 4, 0, 2, 0, 0, 4, 0, 0,
       0, 2, 2, 2, 2, 2, 2, 8, 2, 0, 8, 0, 0, 0, 4, 0, 0, 2, 0, 0, 2, 0,
       0, 4, 2, 0, 4, 6, 2, 2, 2, 2, 0, 0, 2, 2, 4, 0, 0, 0, 2, 0, 2, 0,
       0, 2, 2, 2, 0, 2, 0, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0,
       0, 2, 2, 2, 2, 0, 0, 2, 0, 4, 0, 2, 2, 0, 2, 0, 0, 2, 2, 4, 0, 0,
       2, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 4,
       2, 0, 2, 0, 0, 2, 2, 0, 2, 0, 2, 2, 0, 4, 2,